<a href="https://colab.research.google.com/github/alex-jk/painting-lora-finetune/blob/main/paintings_lora_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!wget https://raw.githubusercontent.com/alex-jk/painting-lora-finetune/main/data/paintings.zip -O paintings.zip
!unzip -o paintings.zip -d paintings/

--2025-04-07 22:43:52--  https://raw.githubusercontent.com/alex-jk/painting-lora-finetune/main/data/paintings.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7164316 (6.8M) [application/zip]
Saving to: ‘paintings.zip’

paintings.zip       100%[===================>]   6.83M  --.-KB/s    in 0.06s   

2025-04-07 22:43:52 (107 MB/s) - ‘paintings.zip’ saved [7164316/7164316]

Archive:  paintings.zip
   creating: paintings/paintings/
  inflating: paintings/paintings/17030-islands-on-kabenung-40x60-brigitte-granton_orig.jpeg  
  inflating: paintings/paintings/23060-Setting-in-a-Grand-Way-Grand-Lake-30X60_-3240.jpeg  
  inflating: paintings/paintings/24055-Sunswept-Opeongo-40x48_-oil-Algonquin-G-539x443.jpeg  
  inflating: paintings/paintings/30yl2012ivanhoethec

In [3]:
!pip install xformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 69.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [4]:
!pip uninstall -y diffusers
!pip install git+https://github.com/huggingface/diffusers.git

Found existing installation: diffusers 0.32.2
Uninstalling diffusers-0.32.2:
  Successfully uninstalled diffusers-0.32.2
  Cloning https://github.com/huggingface/diffusers.git to /tmp/pip-req-build-xoo5zgip
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers.git /tmp/pip-req-build-xoo5zgip
  Resolved https://github.com/huggingface/diffusers.git to commit 506f39af3a7b533209cc96f1732fff347070bdbd
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.33.0.dev0-py3-none-any.whl size=3544299 sha256=3603ef0ecdcb275c0792cd5ced8c31e14abc3bd5bbc7d2d6c004793106dff9fe
  Stored in directory: /tmp/pip-ephem-wheel-cache-yumqx2rg/wheels/d2/5c/5f/16639722ea17ecb73ab461b81718584bac08af2801619786b9
Successfully built diffusers


In [23]:
import diffusers
print(diffusers.__version__)

from diffusers import StableDiffusionPipeline, UNet2DConditionModel
import torch

from peft import LoraConfig # TaskType is no longer needed here
import torch.optim as optim

from torchvision import transforms # Import torchvision for image transforms
from torch.utils.data import Dataset, DataLoader # Import Dataset and DataLoader
from PIL import Image # Import PIL for image loading
import os # Import os for path manipulation
import glob
import subprocess # Better way to run shell commands

0.33.0.dev0


In [15]:
# Load pipeline
pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16,
    safety_checker=None
).to("cuda")

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


In [16]:
# Enable LoRA training in the U-Net
pipe.unet.enable_gradient_checkpointing()
pipe.enable_xformers_memory_efficient_attention()

In [17]:
# Get a reference to the UNet model
unet = pipe.unet
print("UNet reference obtained.")
# 2. Define the LoRA configuration object
#    Omitting task_type as it's not required for unet.add_adapter()
#    and the previous enum value was incorrect.
lora_config = LoraConfig(
    r=4,
    lora_alpha=4,
    target_modules=["to_q", "to_k", "to_v", "to_out.0"],
    lora_dropout=0.05,
    bias="none",
    # task_type=... # REMOVED - Not needed here and caused AttributeError
)

print("LoRA Config defined (without task_type).")

UNet reference obtained.
LoRA Config defined (without task_type).


In [18]:
# 3. Add the LoRA adapter to the UNet model using the defined config
#    The 'adapter_name' is optional but good practice if you plan
#    to manage multiple adapters later.
adapter_name = "my_style_lora" # You can choose any descriptive name
unet.add_adapter(lora_config, adapter_name=adapter_name)

print(f"LoRA adapter '{adapter_name}' added to the UNet.")

# Optional but recommended: Verify trainable parameters
# This helps confirm that only the LoRA weights are set to be trained.
trainable_params = 0
all_param = 0
for name, param in unet.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
        # Print trainable parameter names (optional, for debugging)
        # print(f"Trainable: {name}")
        trainable_params += param.numel()

print(
    f"\nTotal UNet params: {all_param:,}\n"
    f"Trainable params (LoRA): {trainable_params:,}\n"
    f"Trainable %: {100 * trainable_params / all_param:.4f}%"
)

LoRA adapter 'my_style_lora' added to the UNet.

Total UNet params: 860,318,148
Trainable params (LoRA): 797,184
Trainable %: 0.0927%


In [24]:
# 4. Set up the Optimizer
#    Filter the UNet's parameters to get only those that require gradients (the LoRA parameters).
lora_params_to_optimize = [p for p in unet.parameters() if p.requires_grad]
print(f"Number of parameter groups to optimize: {len(lora_params_to_optimize)}")

# Define common hyperparameters for the optimizer
learning_rate = 1e-4 # A common starting LR for LoRA training
adam_beta1 = 0.9
adam_beta2 = 0.999
adam_weight_decay = 1e-2
adam_epsilon = 1e-08

# Create the AdamW optimizer, passing only the LoRA parameters
optimizer = optim.AdamW(
    lora_params_to_optimize,
    lr=learning_rate,
    betas=(adam_beta1, adam_beta2),
    weight_decay=adam_weight_decay,
    eps=adam_epsilon,
)

print(f"Optimizer AdamW created with learning rate {learning_rate}.")

Number of parameter groups to optimize: 256
Optimizer AdamW created with learning rate 0.0001.


In [25]:
# Get references to VAE, Text Encoder, Tokenizer, and Scheduler from the pipeline
vae = pipe.vae
text_encoder = pipe.text_encoder
tokenizer = pipe.tokenizer
scheduler = pipe.scheduler # The noise scheduler

print("VAE, Text Encoder, Tokenizer, and Scheduler extracted from pipeline.")

# Freeze VAE and Text Encoder parameters
# These models should not be trained during LoRA fine-tuning of the UNet
vae.requires_grad_(False)
text_encoder.requires_grad_(False)

print("VAE and Text Encoder parameters frozen (requires_grad=False).")

# Optional: Move VAE and Text Encoder to appropriate dtype and device if not already done
# (The pipe.to("cuda") should have handled this, but explicit check/move can be useful)
# vae.to(unet.device, dtype=torch.float16) # Match UNet's device and potentially dtype
# text_encoder.to(unet.device) # Text encoder usually stays float32

# Verify UNet is still in training mode (add_adapter should preserve this)
if not unet.training:
    unet.train()
    print("Set UNet back to training mode.")
else:
    print("UNet is already in training mode.")

VAE, Text Encoder, Tokenizer, and Scheduler extracted from pipeline.
VAE and Text Encoder parameters frozen (requires_grad=False).
UNet is already in training mode.


In [26]:
# --- Code just for checking the directory ---

# Define the directory where files were supposed to be unzipped
base_data_dir = "paintings"
possible_subdir = os.path.join(base_data_dir, "paintings") # Common pattern if zip contained a 'paintings' folder

print(f"--- Checking contents of '{base_data_dir}' ---")

# Check if the base directory exists
if not os.path.isdir(base_data_dir):
    print(f"Error: Directory '{base_data_dir}' does not exist.")
else:
    # List immediate contents
    print(f"\nDirect contents of '{base_data_dir}':")
    try:
        for item in os.listdir(base_data_dir):
            item_path = os.path.join(base_data_dir, item)
            item_type = "DIR" if os.path.isdir(item_path) else "FILE"
            print(f"- {item} ({item_type})")
    except Exception as e:
        print(f"Error listing contents: {e}")


    # Look for image files directly inside base_data_dir
    image_extensions = ('*.png', '*.jpg', '*.jpeg', '*.webp')
    images_in_base = []
    for ext in image_extensions:
        images_in_base.extend(glob.glob(os.path.join(base_data_dir, ext)))

    if images_in_base:
        print(f"\nFound {len(images_in_base)} image(s) directly in '{base_data_dir}'. Example: {os.path.basename(images_in_base[0])}")
    else:
        print(f"\nNo image files found directly in '{base_data_dir}'.")

    # Check if the common subdirectory exists and look for images there
    if os.path.isdir(possible_subdir):
        print(f"\nChecking common subdirectory '{possible_subdir}'...")
        images_in_subdir = []
        for ext in image_extensions:
            images_in_subdir.extend(glob.glob(os.path.join(possible_subdir, ext)))

        if images_in_subdir:
            print(f"Found {len(images_in_subdir)} image(s) in subdirectory '{possible_subdir}'. Example: {os.path.basename(images_in_subdir[0])}")
        else:
            print(f"No image files found in subdirectory '{possible_subdir}'.")
    else:
         print(f"\nCommon subdirectory '{possible_subdir}' does not exist.")


print("\n--- End Check ---")

--- Checking contents of 'paintings' ---

Direct contents of 'paintings':
- paintings (DIR)

No image files found directly in 'paintings'.

Checking common subdirectory 'paintings/paintings'...
Found 24 image(s) in subdirectory 'paintings/paintings'. Example: the-evening-hour-jake-vandenbrink.jpg

--- End Check ---


In [21]:
# Download, Extract Data, and Prepare Dataset/DataLoader

# Define data source and target directory
data_url = "https://raw.githubusercontent.com/alex-jk/painting-lora-finetune/main/data/paintings.zip"
zip_file = "paintings.zip"
data_dir = "paintings" # Directory where data will be unzipped

# Create target directory if it doesn't exist
os.makedirs(data_dir, exist_ok=True)

# Download the data if zip file doesn't exist
if not os.path.exists(zip_file):
    print(f"Downloading data from {data_url}...")
    try:
        # Using subprocess for better error handling than '!' in some environments
        subprocess.run(["wget", data_url, "-O", zip_file], check=True)
        print("Download complete.")
    except subprocess.CalledProcessError as e:
        print(f"Error downloading file: {e}")
        # Handle error appropriately, maybe raise exception
        raise SystemExit(f"Failed to download data: {e}")
    except FileNotFoundError:
        print("Error: 'wget' command not found. Please ensure wget is installed.")
        raise SystemExit("wget not found.")
else:
    print(f"Zip file '{zip_file}' already exists. Skipping download.")

# Unzip the data
print(f"Unzipping {zip_file} into {data_dir}...")
try:
    # Use -o to overwrite existing files without prompting
    subprocess.run(["unzip", "-o", zip_file, "-d", data_dir], check=True, capture_output=True)
    print("Unzipping complete.")
except subprocess.CalledProcessError as e:
    print(f"Error unzipping file: {e}")
    print(f"Stderr: {e.stderr.decode()}")
    raise SystemExit(f"Failed to unzip data: {e}")
except FileNotFoundError:
    print("Error: 'unzip' command not found. Please ensure unzip is installed.")
    raise SystemExit("unzip not found.")

Zip file 'paintings.zip' already exists. Skipping download.
Unzipping paintings.zip into paintings...
Unzipping complete.


In [1]:
# Define Hyperparameters for data preparation
image_resolution = 512 # Match the resolution SD v1.5 was trained on
batch_size = 4       # Adjust based on your GPU memory (start small)

# Define Image Transformations
preprocess = transforms.Compose(
    [
        transforms.Resize(image_resolution, interpolation=transforms.InterpolationMode.BILINEAR),
        transforms.CenterCrop(image_resolution),
        transforms.ToTensor(), # Convert image to tensor [0, 1]
        transforms.Normalize([0.5], [0.5]), # Normalize to [-1, 1]
    ]
)

# Define Custom Dataset pointing to the 'paintings' directory
class PaintingStyleDataset(Dataset):
    def __init__(self, instance_data_root, tokenizer, size=512):
        self.instance_data_root = instance_data_root
        self.tokenizer = tokenizer # Keep tokenizer reference if needed later
        self.size = size

        self.instance_images_path = []
        valid_extensions = ('.png', '.jpg', '.jpeg', '.webp')
        if not os.path.isdir(instance_data_root):
             raise ValueError(f"Dataset directory '{instance_data_root}' not found after unzipping.")

        for filename in os.listdir(instance_data_root):
            if filename.lower().endswith(valid_extensions):
                self.instance_images_path.append(os.path.join(instance_data_root, filename))

        self.num_instance_images = len(self.instance_images_path)
        self._length = self.num_instance_images
        self.image_transforms = preprocess

        if self.num_instance_images == 0:
             print(f"Warning: No valid image files found in {instance_data_root}. Check the directory structure and file extensions.")
        else:
            print(f"Dataset initialized: Found {self.num_instance_images} images in {instance_data_root}")


    def __len__(self):
        return self._length

    def __getitem__(self, index):
        example = {}
        if index >= self.num_instance_images: # Should not happen with DataLoader but safe check
             raise IndexError("Index out of bounds")

        instance_image_path = self.instance_images_path[index]
        try:
            instance_image = Image.open(instance_image_path)
            if not instance_image.mode == "RGB":
                instance_image = instance_image.convert("RGB")
            example["instance_images"] = self.image_transforms(instance_image)
        except Exception as e:
            print(f"Warning: Could not load image {instance_image_path}. Error: {e}. Skipping index {index}.")
            # Return None to indicate failure, handled in collate_fn
            return None

        # Define the text prompt for training.
        # Since we're training a style, use a consistent prompt with a trigger word.
        # <<< IMPORTANT: Replace 'sks painting style' with your desired trigger phrase >>>
        instance_prompt = "a painting in sks painting style"
        example["instance_prompt"] = instance_prompt # Store the raw text prompt

        return example

# Define the Collate Function (handles potential None values from failed image loads)
def simple_collate_fn(examples):
    # Filter out None entries (failed image loads)
    valid_examples = [ex for ex in examples if ex is not None]
    if not valid_examples: # Handle case where entire batch failed
        return None # Or however your training loop handles empty batches

    # Stack images
    pixel_values = torch.stack([example["instance_images"] for example in valid_examples])
    pixel_values = pixel_values.to(memory_format=torch.contiguous_format).float()

    # Collect prompts
    prompts = [example["instance_prompt"] for example in valid_examples]

    return {"pixel_values": pixel_values, "prompts": prompts}

NameError: name 'transforms' is not defined

In [22]:
actual_image_dir = "paintings/paintings"

# Instantiate the Dataset using the 'data_dir' ('paintings')
train_dataset = PaintingStyleDataset(
    instance_data_root=data_dir,
    tokenizer=tokenizer,
    size=image_resolution
)

# Create the DataLoader only if the dataset has images
if len(train_dataset) > 0:
    train_dataloader = DataLoader(
        train_dataset,
        shuffle=True,
        collate_fn=simple_collate_fn,
        batch_size=batch_size,
        num_workers=2 # Adjust based on your system
    )
    print(f"DataLoader created with batch size {batch_size}.")

    # Optional: Test the dataloader
    # try:
    #     first_batch = next(iter(train_dataloader))
    #     if first_batch:
    #         print("First batch keys:", first_batch.keys())
    #         print("Image batch shape:", first_batch['pixel_values'].shape)
    #         print(f"Number of prompts in batch: {len(first_batch['prompts'])}")
    #     else:
    #         print("First batch was empty (all image loads failed?).")
    # except StopIteration:
    #     print("DataLoader is empty (dataset likely has zero length).")

else:
    print("DataLoader not created because the dataset is empty.")
    train_dataloader = None # Ensure it's None if not created

DataLoader not created because the dataset is empty.
